In [1]:
from pyspark.sql.functions import countDistinct
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
import json

In [20]:
#reading in data again to get the demographics 
df_new = spark.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/transposed/BBM-STICKER-SEND/")
df_demo = spark.read.parquet("gs://ds-user-demographic/parquet/tastes_mau_1024.parquet")

In [5]:
#join to derive features
from pyspark.sql.functions import col
active_base  = df_new.alias('a').join(df_demo.alias('b'),col('b.regid') == col('a.user_id_n')).select([col('time'),col('City'),col('Connection'),col('Country'),col('Device_Brand'),col('Device_Model'),col('New_User'),col('OS_Version'),col('app_name'),col('carrier'),col('license_type'),col('a.platform'),col('sticker_id'),col('sticker_pack_id'),col('user_id_n')]+[col('language'),col('age_group'),col('gender')])

In [4]:
#stickertypeprocess

df_free = sc.textFile("gs://ds-url-catag/Stickers/sticker_data_tempfiles/Sticker_free.csv") \
    .map(lambda line: line.split(";")).toDF(['Sticker_id', 'sticker_name', 'license_type'])
    
df_paid = sc.textFile("gs://ds-url-catag/Stickers/sticker_data_tempfiles/Stickers_paid.csv") \
    .map(lambda line: line.split(";")).toDF(['Sticker_id', 'sticker_name', 'license_type'])
    
df_new = df_free.union(df_paid)

In [6]:
df_new.show()

+------------------+--------------------+------------+
|        Sticker_id|        sticker_name|license_type|
+------------------+--------------------+------------+
|candyshoppingspree|Candy Shopping Spree|        FREE|
|       draculaalay|        Dracula Alay|        FREE|
|           ceblong|             Ceblong|        FREE|
|sinetronanakmasjid|Sinetron Anak Masjid|        FREE|
|     sinetrontuhan|Sinetron Tuhan Ad...|        FREE|
|       sinetrondia|        Sinetron Dia|        FREE|
|         roundmons|           Roundmons|        FREE|
|       beardieboys|    The Beardie Boys|        FREE|
|        slangwords|         SLANG WORDS|        FREE|
|      bbmhalloween|           Halloween|        FREE|
|          icecream|           Ice Cream|        FREE|
|           pumpkin|             Pumpkin|        FREE|
| halloweenstickers|           Halloween|        FREE|
|      lagilagigila|      Lagi Lagi Gila|        FREE|
|     letsgetmoving|    Let's Get Moving|        FREE|
|   itstar

In [2]:
check = spark.read.parquet("gs://ds-url-catag/Stickers/stick_bytype/agg_proc_stickertypes/")

In [3]:
check.show(5000)

+--------------------+----+
|                  id|type|
+--------------------+----+
|          wallschoco|FREE|
|       bingewatching|FREE|
|       wonderwomanew|FREE|
|   pepsodentramadhan|FREE|
|        kfcindonesia|FREE|
|               bulan|FREE|
|    bulanpenuhberkah|FREE|
|       ramadhanceria|FREE|
|       salamramadhan|FREE|
|       pesanramadhan|FREE|
|       happyramadhan|FREE|
|ramadanceriastickers|FREE|
|   bbmfamilyramadhan|FREE|
|        rinduramadan|FREE|
|         polisibaper|FREE|
|      denbagusedarso|FREE|
|                olly|FREE|
|  angkringanmasgiman|FREE|
|       emakgalakfree|FREE|
|     bouquetemotions|FREE|
|           chickyboy|FREE|
|           letsgoout|FREE|
|       ikasicewegaul|FREE|
|          bbmfunsize|FREE|
|            jakmania|FREE|
|      cottonvesakday|FREE|
|       letteringindo|FREE|
|        sacintapapua|FREE|
|             chickun|FREE|
|    pixelboystickers|FREE|
|        ekspresisaja|FREE|
|       muslimahcilik|FREE|
|           jowocada

In [4]:
df = sc.textFile("gs://ds-url-catag/Stickers/stick_bytype/all_stickers_feb20/").map(lambda line: line.split(","))
# .map(lambda line: (line[0],line[1])).toDF(['id','type'])

In [5]:
df.take(10)

[[u'"\\"bbmwintersports\\"', u'\\"FREE\\""'],
 [u'"\\"womensdaybbm\\"', u'\\"FREE\\""'],
 [u'"\\"cottonandfamilyvacation\\"', u'\\"FREE\\""'],
 [u'"\\"peterandpetrastory\\"', u'\\"FREE\\""'],
 [u'"\\"lalafriends\\"', u'\\"FREE\\""'],
 [u'"\\"moko\\"', u'\\"FREE\\""'],
 [u'"\\"zimkhitha\\"', u'\\"FREE\\""'],
 [u'"\\"cottonandfamilyworklife\\"', u'\\"FREE\\""'],
 [u'"\\"snowman\\"', u'\\"FREE\\""'],
 [u'"\\"chacha\\"', u'\\"FREE\\""']]

In [14]:
# tets_base = active_base.alias('a').join(df_new,df_new.Sticker_id == active_base.sticker_pack_id ).select([active_base.sticker_pack_id ])

In [ ]:
#write the final_file
